## Оценка генерации стихов предобученной моделью ruGPT

In [ ]:
!pip install transformers

In [2]:
import torch
import math
import csv
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from scipy.special import softmax

In [3]:
from google.colab import drive
drive.mount("/content/drive")

%cd /content/drive/MyDrive/NLP_poems

Mounted at /content/drive
/content/drive/MyDrive/NLP_poems


In [4]:
MODEL_NAME = 'sberbank-ai/rugpt2large'
DEVICE = 'cuda'

model = GPT2LMHeadModel.from_pretrained(MODEL_NAME).to(DEVICE)
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

In [21]:
def gpt_evaluate(text, model, tokenizer):
    text = " ".join(text.split())
    tokenize_input = tokenizer.tokenize(text)
    tokenizer.convert_tokens_to_ids(tokenize_input)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)], device=DEVICE)
    labels = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)], device=DEVICE)
    labels[:,:1] = -1
    loss = model(tensor_input, labels=tensor_input)
    return float(loss[0]) * -1.0

In [ ]:
model.eval()

def evaluate_data(poems_path, output_path):
    data = pd.read_csv(poems_path,  header=None,delimiter='\t', quotechar='|', usecols=[1])
    data['gpt_lp'] = data[1].apply(lambda x: gpt_evaluate(x, model, tokenizer))
    data.sort_values(by=['gpt_lp'], axis=0, ascending=False, inplace=True)
    data.rename({1:'poem'}, axis=1,  inplace=True)
    data.to_csv(output_path, index=False)

In [22]:
poems_path = 'generated_poems/lstm_acc_poems.csv'
output_path = 'generated_poems/with_lp/lstm_acc_poems_lp.csv'

evaluate_data(poems_path, output_path)

In [29]:
poems_path = 'generated_poems/markov_acc_civil.csv'
output_path = 'generated_poems/with_lp/markov_acc_civil_lp.csv'

evaluate_data(poems_path, output_path)

In [30]:
poems_path = 'generated_poems/markov_acc_love.csv'
output_path = 'generated_poems/with_lp/markov_acc_love_lp.csv'

evaluate_data(poems_path, output_path)

In [31]:
poems_path = 'generated_poems/markov_acc_nature.csv'
output_path = 'generated_poems/with_lp/markov_acc_nature_lp.csv'

evaluate_data(poems_path, output_path)

In [32]:
poems_path = 'generated_poems/markov_acc_religion.csv'
output_path = 'generated_poems/with_lp/markov_acc_religion_lp.csv'

evaluate_data(poems_path, output_path)